In [ ]:
# Ler arquivos

# estrutura dos dados
# Audio Cats and Dogs/cats_dogs/
# test
#   cat
#   test (dog)
# train
#   cat
#   dog

# file format .wav


from glob import glob
from pathlib import Path

# Folder paths
cat_train_folder_path = '../input/audio-cats-and-dogs/cats_dogs/train/cats'
cat_test_folder_path = '../input/audio-cats-and-dogs/cats_dogs/test/cats'

dog_train_folder_path = '../input/audio-cats-and-dogs/cats_dogs/train/dogs'
dog_test_folder_path = '../input/audio-cats-and-dogs/cats_dogs/test/cats'

# File format regex pattern
file_format_pattern =  '*.wav'

# File paths
cat_train_file_paths = glob(os.path.join(cat_train_folder_path, file_format_pattern))
cat_test_file_paths = glob(os.path.join(cat_test_folder_path, file_format_pattern))

dog_train_file_paths = glob(os.path.join(dog_train_folder_path, file_format_pattern))
dog_test_file_paths = glob(os.path.join(dog_test_folder_path, file_format_pattern))


In [ ]:
# Classifica o dataset (cat=0 and dog=1)
train_data = [{'path': path, 'label': 0} for path in cat_train_file_paths] + \
             [{'path': path, 'label': 1} for path in dog_train_file_paths]

test_data = [{'path': path, 'label': 0} for path in cat_test_file_paths] + \
            [{'path': path, 'label': 1} for path in dog_test_file_paths]


train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

print(train_df.head())
print(test_df.head())

In [ ]:
import librosa
# Inicializa a lista para coletar dados
data_train = []
data_test = []

# Define uma função para extrair características
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Em vez de tomar a média, você pode querer transpor os MFCCs para que 
    # a dimensão temporal seja a primeira dimensão
    mfccs = np.transpose(mfccs, [1, 0])
    return mfccs

# Itera sobre os arquivos de gato e cachorro e extrai as características
for file_paths, label in zip([cat_train_file_paths, dog_train_file_paths], [0, 1]):
    for file_path in file_paths:
        features = extract_features(file_path)
        data_train.append([features, label]) 

for file_paths, label in zip([cat_test_file_paths, dog_test_file_paths], [0, 1]):
    for file_path in file_paths:
        features = extract_features(file_path)
        data_test.append([features, label]) 


In [ ]:
# Adaptação do código para preencher os arrays X e y
X_train = []  # Features
y_train = []  # Labels

# Adaptação do código para preencher os arrays X e y
X_test = []  # Features
y_test = []  # Labels

for features, label in data_train:
    if features is not None:
        X.append(features)
        y.append(label)

X_train = np.array(X)
y_train = np.array(y)

for features, label in data_test:
    if features is not None:
        X.append(features)
        y.append(label)

X_test = np.array(X)
y_test = np.array(y)

In [ ]:
# Criar modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import librosa
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(altura_espectrograma, largura_espectrograma, canais)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # ou 'softmax' para classificação multiclasse
])

# Compilar o modelo
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=10)


In [ ]:
# Testar o modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')